In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from synthetic import generate_ds1
from kan import KAN

In [ ]:
lag = 4  # number of lags
T = 1000  # number of time points
num_series = 3  # number of time series

batch_size = 32
epochs = 20
lr = 1e-3
sparsity = 0.8
max_iter = 30

dim_input = num_series * lag
dim_output = 1

width = [dim_input, 16, dim_output]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
X_np = generate_ds1(sig_to_noise=100)
X_np

In [ ]:
# Plot data
for i in range(len(X_np.columns)):
    plt.plot(X_np.index, X_np.iloc[:, i], label='Series {}'.format(X_np.columns[i]))
plt.legend()
plt.show()

In [ ]:
# transfer dataframe to tensor 
X_np = torch.tensor(X_np.values, dtype=torch.float32).to(device)

# add one new dimension to the tensor
X = X_np.unsqueeze(0)
X.shape

In [ ]:
train = X[:, :, :-2]
y_no_GC = X[:, :, -2]
y_GC = X[:, :, -1]

# rearrrange the data
def rearrange_data(train, y, lag):
    li = []
    for i in range(train.shape[2]):
        for j in range(lag):
            li.append(train[0, :, i].detach().numpy()[j:-lag+j])
    train = torch.tensor(np.array(li).T, dtype=torch.float32).to(device)

    y = y[:, lag:].T
    array = torch.cat((train, y), 1)
    
    return array
    
array_no_GC = rearrange_data(train, y_no_GC, lag)
array_GC = rearrange_data(train, y_GC, lag)
array_no_GC.shape

In [ ]:
def create_dataset(X):
    dataset = {}
    dataset['train_input'] = X[:, :-1]
    dataset['test_input'] = X[:, :-1]
    dataset['train_label'] = X[:, -1]
    dataset['test_label'] = X[:, -1]

    return dataset

In [ ]:
ds_GC = create_dataset(array_no_GC)
ds_no_GC = create_dataset(array_GC)

## Build KAN for Dataset with GC

In [ ]:
model1 = KAN(width=width, grid=3, k=3, seed=42)
loss1 = model1.fit(ds_GC, opt='LBFGS', steps=max_iter, lamb=0.001)['train_loss']

# plot loss1
plt.plot(loss1)
plt.show()


In [ ]:
# model1.plot()

In [ ]:
def get_ts_scores(model):
    ts_scores = model.node_scores[0].view(3, lag).sum(dim=1).detach().numpy()
    return ts_scores

## Build KAN for Dataset with GC

In [ ]:
model2 = KAN(width=width, grid=3, k=3, seed=42)
loss2 = model2.fit(ds_no_GC, opt='LBFGS', steps=max_iter, lamb=0.001)['train_loss']

# plot loss2
plt.plot(loss2)
plt.show()

In [ ]:
# model2.plot()

In [ ]:
print('GC score for model1:', get_ts_scores(model1))
print('GC score for model2:', get_ts_scores(model2))